In [6]:
import pandas as pd 
from sklearn.datasets import make_moons

# Erzeugung künstlicher Daten
X_array, y_array = make_moons(n_samples=120, random_state=0, noise=0.3)

daten = pd.DataFrame({
    'Kilometerstand [km]': 10000 * (X_array[:,0] + 2),
    'Preis [EUR]': 5000 * (X_array[:,1] + 2),
    'verkauft': y_array,
    })

# Adaption der Daten
X = daten[['Kilometerstand [km]', 'Preis [EUR]']].values
y = daten['verkauft'].values

In [7]:
from sklearn.ensemble import RandomForestClassifier

model_random_forest = RandomForestClassifier(n_estimators=4, random_state=0)
model_random_forest.fit(X,y)

RandomForestClassifier(n_estimators=4, random_state=0)

In [8]:
model_random_forest.feature_importances_

array([0.37127102, 0.62872898])

In [19]:
import numpy as np 

f0_list = []
f1_list = []
for i in range(4):
    f0, f1 = model_random_forest.estimators_[i].feature_importances_
    f0_list.append(f0)
    f1_list.append(f1)

print(f0_list)
print(f1_list)

[0.4798267641118992, 0.16422007399451005, 0.338736844169486, 0.5023004079148654]
[0.5201732358881008, 0.83577992600549, 0.661263155830514, 0.49769959208513465]
